In [2]:
import pandas as pd
import numpy as np

kevin = pd.read_pickle('Download/Cleaned Data with Longformer/StoriesAboutKevin.csv.pkl')
creepy = pd.read_pickle('Download/Cleaned Data with Longformer/CreepyPasta.csv.pkl')
creepy = creepy[:len(kevin)].copy()

In [15]:
from textblob import TextBlob

In [16]:
zen = TextBlob(kevin.loc[4,'selftext'])

In [17]:
zen.sentences

[Sentence("I was living with my older brother while attending college and stayed up one night to watch the lunar eclipse."),
 Sentence("As I rushed outside my brother asked where I was going in such a rush."),
 Sentence("I told him I wanted to see the lunar eclipse."),
 Sentence("He the asked (and I quote), "Is that when the sun goes in front of the moon?""),
 Sentence("He wasn't always a Kevin."),
 Sentence("He was a hippie in the Sixties and just continued to drink and drug well past the point of idiocy."),
 Sentence("I've seen him nail his hand to a wall, walk off of a loading dock and face plant, stab himself in the knee with a screwdriver, loose the ability to read and write over time."),
 Sentence("I'm not sure a self-made Kevin counts but there it is.")]

In [18]:
for sentence in zen.sentences:
    print(sentence.sentiment)

Sentiment(polarity=0.16666666666666666, subjectivity=0.3333333333333333)
Sentiment(polarity=0.0, subjectivity=0.275)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=-0.275, subjectivity=0.325)
Sentiment(polarity=-0.07692307692307693, subjectivity=0.2692307692307692)
Sentiment(polarity=-0.25, subjectivity=0.8888888888888888)


In [46]:
zen.sentiment_assessments

Sentiment(polarity=-0.10146520146520145, subjectivity=0.38449328449328446, assessments=[(['older'], 0.16666666666666666, 0.3333333333333333, None), (['outside'], 0.0, 0.05, None), (['such'], 0.0, 0.5, None), (['past'], -0.25, 0.25, None), (['idiocy'], -0.3, 0.4, None), (['loose'], -0.07692307692307693, 0.2692307692307692, None), (['not', 'sure'], -0.25, 0.8888888888888888, None)])

In [43]:
tuple(list(zen.sentiment_assessments)[:2])

(-0.10146520146520145, 0.38449328449328446)

In [54]:
def blob_senti_assessments(text):
    zen = TextBlob(text)
    polarity, subjectivity = tuple(zen.sentiment_assessments)[:2]
    return polarity, subjectivity

creepy['polarity'], creepy['subjectivity'] = zip(*creepy['selftext'].map(blob_senti_assessments))
kevin['polarity'], kevin['subjectivity'] = zip(*kevin['selftext'].map(blob_senti_assessments))

In [50]:
import tensorflow as tf

In [51]:
creepyLF = tf.stack(creepy.loc[:,'LF pooler output'].to_list())
kevinLF = tf.stack(kevin.loc[:,'LF pooler output'].to_list())

creepyLF = tf.reshape(creepyLF, creepyLF.shape[::2])
kevinLF = tf.reshape(kevinLF, kevinLF.shape[::2])

In [55]:
import numpy as np
from sklearn.manifold import TSNE
import seaborn as sns

%matplotlib widget
import matplotlib.pyplot as plt

X_embedded_nosleep = TSNE(perplexity=30, learning_rate=50, n_components=3, n_iter=5000).fit_transform(creepyLF)
X_embedded_self = TSNE(perplexity=30, learning_rate=50, n_components=3, n_iter=2000).fit_transform(kevinLF)

df1 = pd.DataFrame(X_embedded_nosleep, columns=['x','y', 'z'])
df1['subreddit'] = 'creepy'
df1[['polarity','subjectivity']] = creepy[['polarity','subjectivity']]
df2 = pd.DataFrame(X_embedded_self, columns=['x','y', 'z'])
df2['subreddit'] = 'kevin'
df2[['polarity','subjectivity']] = kevin[['polarity','subjectivity']]

df = pd.concat([df1,df2])

In [56]:
df

,x,y,z,subreddit,polarity,subjectivity
0,-13.846015,-30.075722,19.178976,creepy,0.500000,0.888889
1,-1.835241,-0.122834,16.612370,creepy,-0.156250,0.375000
2,-1.686234,-12.288081,41.553974,creepy,0.075000,0.400000
3,-5.652670,3.080822,17.556034,creepy,-0.026020,0.429932
4,-7.979607,-6.499038,14.884555,creepy,0.459615,0.532692
...,...,...,...,...,...,...
3398,-5.209599,0.949645,12.987222,kevin,-0.009167,0.492778
3399,-17.569633,-17.486656,-4.802795,kevin,0.455000,0.280000
3400,-11.678261,17.448963,10.173971,kevin,-0.247966,0.559127
3401,2.363497,27.583939,21.587803,kevin,-0.001716,0.413725


In [57]:
import matplotlib.cm as cm

creepy_colors = cm.rainbow(np.linspace(0, 1, 4))
kevin_colors = cm.rainbow(np.linspace(0, 1, 4))

In [65]:
df.T.values

array([[-13.846014976501465, -1.8352407217025757, -1.6862335205078125,
        ..., -11.678260803222656, 2.363496780395508, 4.659627437591553],
       [-30.075721740722656, -0.12283439934253693, -12.288081169128418,
        ..., 17.448963165283203, 27.583938598632812, 3.7279324531555176],
       [19.17897605895996, 16.612369537353516, 41.55397415161133, ...,
        10.173971176147461, 21.58780288696289, 18.52151870727539],
       ['creepy', 'creepy', 'creepy', ..., 'kevin', 'kevin', 'kevin'],
       [0.5, -0.15625, 0.07500000000000001, ..., -0.24796626984126982,
        -0.0017156862745098076, -0.08900793650793652],
       [0.8888888888888888, 0.375, 0.4, ..., 0.5591269841269841,
        0.41372549019607846, 0.2968650793650794]], dtype=object)

In [71]:
from mpl_toolkits.mplot3d import Axes3D,axes3d
sns.set(style = "darkgrid")



fig = plt.figure()
fig.set_size_inches(18.5, 10.5)
ax = fig.add_subplot(111, projection = '3d')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

ax.scatter(df[(df['subreddit']=='creepy') & (df['polarity']>0.5) & (df['subjectivity']>0.5)]['x'], 
           df[(df['subreddit']=='creepy') & (df['polarity']>0.5) & (df['subjectivity']>0.5)]['y'], 
           df[(df['subreddit']=='creepy') & (df['polarity']>0.5) & (df['subjectivity']>0.5)]['z'], 
           c='brown', label='CreepyPasta (p&s > 0.5)')
ax.scatter(df[(df['subreddit']=='creepy') & (df['polarity']<0.5) & (df['subjectivity']>=0.5)]['x'], 
           df[(df['subreddit']=='creepy') & (df['polarity']<0.5) & (df['subjectivity']>=0.5)]['y'], 
           df[(df['subreddit']=='creepy') & (df['polarity']<0.5) & (df['subjectivity']>=0.5)]['z'], 
           c='red', label='CreepyPasta (p<0.5, s>0.5)', marker='^')
ax.scatter(df[(df['subreddit']=='creepy') & (df['polarity']>=0.5) & (df['subjectivity']<0.5)]['x'], 
           df[(df['subreddit']=='creepy') & (df['polarity']>=0.5) & (df['subjectivity']<0.5)]['y'], 
           df[(df['subreddit']=='creepy') & (df['polarity']>=0.5) & (df['subjectivity']<0.5)]['z'], 
           c='coral', label='CreepyPasta (p>0.5, s<0.5)', marker='s')
ax.scatter(df[(df['subreddit']=='creepy') & (df['polarity']<=0.5) & (df['subjectivity']<=0.5)]['x'], 
           df[(df['subreddit']=='creepy') & (df['polarity']<=0.5) & (df['subjectivity']<=0.5)]['y'], 
           df[(df['subreddit']=='creepy') & (df['polarity']<=0.5) & (df['subjectivity']<=0.5)]['z'], 
           c='chocolate', label='CreepyPasta (p&s<0.5)', marker='P')

ax.scatter(df[(df['subreddit']=='kevin') & (df['polarity']>0.5) & (df['subjectivity']>0.5)]['x'], 
           df[(df['subreddit']=='kevin') & (df['polarity']>0.5) & (df['subjectivity']>0.5)]['y'], 
           df[(df['subreddit']=='kevin') & (df['polarity']>0.5) & (df['subjectivity']>0.5)]['z'], 
           c='brown', label='StoriesAboutKevin (p&s > 0.5)')
ax.scatter(df[(df['subreddit']=='kevin') & (df['polarity']<0.5) & (df['subjectivity']>=0.5)]['x'], 
           df[(df['subreddit']=='kevin') & (df['polarity']<0.5) & (df['subjectivity']>=0.5)]['y'], 
           df[(df['subreddit']=='kevin') & (df['polarity']<0.5) & (df['subjectivity']>=0.5)]['z'], 
           c='red', label='StoriesAboutKevin (p<0.5, s>0.5)', marker='^')
ax.scatter(df[(df['subreddit']=='kevin') & (df['polarity']>=0.5) & (df['subjectivity']<0.5)]['x'], 
           df[(df['subreddit']=='kevin') & (df['polarity']>=0.5) & (df['subjectivity']<0.5)]['y'], 
           df[(df['subreddit']=='kevin') & (df['polarity']>=0.5) & (df['subjectivity']<0.5)]['z'], 
           c='coral', label='StoriesAboutKevin (p>0.5, s<0.5)', marker='s')
ax.scatter(df[(df['subreddit']=='kevin') & (df['polarity']<=0.5) & (df['subjectivity']<=0.5)]['x'], 
           df[(df['subreddit']=='kevin') & (df['polarity']<=0.5) & (df['subjectivity']<=0.5)]['y'], 
           df[(df['subreddit']=='kevin') & (df['polarity']<=0.5) & (df['subjectivity']<=0.5)]['z'], 
           c='chocolate', label='StoriesAboutKevin (p&s<0.5)', marker='P')
plt.legend()

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …